In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/MyDrive/eeg_data')
import sys
sys.path.append('/content/drive/MyDrive/eeg_data')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls ./base

Config.py		  ExractFeatures.py	       __pycache__	tensorPCA.py
config_trial_spikes.yalm  load_files_trials_spikes.py  session_data.py	utils.py
eeg.py			  MyRC_ESN.py		       subject.py


In [ ]:
!pip install mne
!pip install yasa
!pip install dotmap
!pip install nilearn
!pip install mne_connectivity
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 28.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.6/423.6 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.2/424.2 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00
  Created wheel for antropy: filename=antropy-0.1.6-py3-none-any.whl size=16878 sha256=5197e917f3b2cb1fc6d725be1344c8a7cbe62691991cd8692da4f6587d662a52
  Stored in directory: /root/.cache/pip/wheels/98/22/06/e91d7bb213c7133d5e2eb34258623e1e19928d5f05e1ee6812
Successfully built antropy
     ━━━━━━

In [ ]:
# @title Listado ficheros datasets creados de los datos raw
import os
import numpy as np

def read_files_and_show (directory = 'dtset'):
    # Verifica si el directorio existe
    if not os.path.exists (directory):
        print(f"El directorio {directory} no existe.")
        return

    # Obtiene la lista de archivos en el subdirectorio
    files = [f for f in os.listdir(directory) if f.endswith('.npy')]

    # Verifica si hay archivos en el directorio
    if not files:
        print(f"No se encontraron archivos .npy en el directorio {directory}.")
        return

    # Lee cada archivo y muestra sus dimensiones
    for file in files:
        path_file = os.path.join (directory, file)
        try:
            data = np.load (path_file)
            print(f"El archivo {file} tiene dimensiones: {data.shape}")
        except Exception as e:
            print(f"No se pudo leer el archivo {file}: {e}")

# Llamada a la función
read_files_and_show ()

El directorio dataset no existe.


In [ ]:
# @title Bayes búsqueda hiperparámetros óptimos

import os
import sys
import optuna
import argparse
import warnings
import numpy as np
import matplotlib.pyplot as plt

from nilearn import plotting

from scipy.stats import norm

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import adjusted_rand_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

# Import specific functions/classes/modules
from base.eeg import (
    all_channels, eeg_channels, eog_channels, process_eeg_data_with_ica,
    process_eeg_data_without_ica
)

from base.MyRC_ESN import (
    MyRC, MyESN
)


def standardize_eeg_signals_opt (X):
    """
    Estandariza las señales de EEG para cada canal de cada sujeto.

    Args:
        X (np.ndarray): Datos de entrada con forma (n_subjects, n_samples, n_channels).

    Returns:
        X_standardized (np.ndarray): Datos estandarizados con la misma forma que X.
    """
    # Copiar X para almacenar los datos estandarizados
    X_standardized = np.copy(X)

    # Función para estandarizar un canal completo
    def standardize_channel (channel_data):
        scaler = StandardScaler ()
        return scaler.fit_transform (channel_data.reshape (-1, 1)).flatten ()

    # Aplicar la estandarización a cada canal para cada sujeto
    for i in range (X.shape [0]):  # Iterar sobre cada sujeto
        X_standardized [i] = np.apply_along_axis (standardize_channel, 0, X [i])

    return X_standardized

def minmax_normalize_eeg_signals_opt (X):
    """
    Normaliza las señales de EEG para cada canal de cada sujeto usando Min-Max.

    Args:
        X (np.ndarray): Datos de entrada con forma (n_subjects, n_samples, n_channels).

    Returns:
        X_normalized (np.ndarray): Datos normalizados con la misma forma que X.
    """
    # Copiar X para almacenar los datos normalizados
    X_normalized = np.copy(X)

    # Función para normalizar un canal completo usando Min-Max
    def normalize_channel (channel_data):
        scaler = MinMaxScaler()
        return scaler.fit_transform (channel_data.reshape (-1, 1)).flatten ()

    # Aplicar la normalización Min-Max a cada canal para cada sujeto
    for i in range (X.shape [0]):  # Iterar sobre cada sujeto
        X_normalized  [i] = np.apply_along_axis (normalize_channel, 0, X [i])

    return X_normalized

    #################
###########################
    #################

def check_and_load_files (channels = 's', ica = True, features = False,
                          freq = 30, decim = 4, verbose = False):

    # Check if the directory exists, if not, raise an exception
    #if not os.path.exists("./dataset"):
        #raise FileNotFoundError("El directorio de datasets no se encuentra disponible.")

     # Name Files EEG data
    file_RC_y = f"./dtset/dt_Younger_{channels}{'_ICA_' if features else '_'}{'features_' if ica else ''}{freq}_{decim}.npy"
    file_RC_o = f"./dtset/dt_Older_{channels}{'_ICA_' if features else '_'}{'features_' if ica else ''}{freq}_{decim}.npy"

    print (f'file_RC_y: {file_RC_y}')
    print (f'file_RC_o: {file_RC_o}')

    # Verificar si los archivos existen
    if not os.path.exists(file_RC_y) or not os.path.exists(file_RC_o):
        raise FileNotFoundError("Los ficheros de datasets no se encuentran disponibles.")

    dataset_RC_y = np.load (file_RC_y)
    dataset_RC_o = np.load (file_RC_o)

    if verbose: print(f'dataset_RC_y: {dataset_RC_y.shape}')
    if verbose: print(f'dataset_RC_o: {dataset_RC_o.shape}')

    # Concatenar los datasets
    dt_classifier = np.concatenate ((dataset_RC_y, dataset_RC_o), axis = 0)

    # Crear etiquetas para cada instancia
    zeros_list = [0] * 23
    ones_list  = [1] * 24

    # Concatenar las dos listas
    dt_labels = np.array(zeros_list + ones_list)

    # Verificar las formas resultantes
    if verbose: print("Forma de los datos concatenados:", dt_classifier.shape)
    if verbose: print("Forma de las etiquetas concatenadas:", dt_labels.shape)

    # Asignar datos y etiquetas
    X = dt_classifier
    Y = dt_labels

    # Define the directory path para registrar los valores de lso hiperparametros y las metricas asociadas
    directory = "./optim_param"
        # Check if the directory exists, if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)

    return X, Y

# Función para ignorar advertencias de tipo FutureWarning
def warn(*args, **kwargs):
    pass

def train_reservoir (config, X, Y, verbose = False):
    # Ignorar advertencias de FutureWarning específicamente
    warnings.filterwarnings ("ignore", category = FutureWarning)

    if verbose: print (f"config:{config}")
    model_clus = MyESN (config)

    X_ = standardize_eeg_signals_opt (X)

    if verbose: print (f'X:{X.shape}')
    if verbose: print (f'X_:{X_.shape}')

    my_rc_clus = MyRC (model_clus, config)
    result_rc  = my_rc_clus.fit (X_)

    # Devolvemos: Estados internos neuronas, Etados internos reducidos a PCA, Representación estados internos, Readout
    rc_state, rc_dim_states, input_repr, output_redout_layer = result_rc

    if verbose: print (f'config:{config}')
    if verbose: print (f'rc_state:{rc_state.shape}')
    if verbose: print (f'rc_dim_states:{rc_dim_states.shape}')
    if verbose: print (f'input_repr:{input_repr.shape}')
    if verbose: print (f'output_redout_layer:{output_redout_layer.shape}')

    return rc_state, input_repr, Y

def calculate_kmeans (X, verbose):

    kmeans = KMeans (n_clusters = 2)
    kmeans.fit_transform (X)
    predicted_labels = kmeans.labels_

    return predicted_labels

def log_results(file_log, trial, config, report, accuracy_value, f1_score, auc_roc_value):
    with open (file_log, "a") as f:
        f.write (f"Trial {trial.number}\n")
        f.write (f"Configuration: {config}\n")
        f.write (f"Report: {report}\n")
        f.write (f"accu: {accuracy_value}\n")
        f.write (f"f1_score: {f1_score}\n")
        f.write (f"auc_roc_value: {auc_roc_value}\n")
        f.write ("\n")


def evaluate_accuracy (trial, X, Y, channels, state, file_prefix, verbose = True):
    """
    Evaluar la precisión de los clústeres utilizando K-means.

    Args:
    - trial: Objeto de prueba para registrar resultados.
    - X: Datos de entrada para calcular clústeres y similitud.
    - Y: Etiquetas verdaderas para comparar con los clústeres predichos.
    - channels: Canales para la evaluación.
    - state: Estado actual del proceso de evaluación.
    - file_prefix: Prefijo para los archivos de registro de resultados.
    - verbose: Booleano para imprimir información detallada.

    Returns:
    - accuracy_value_similarity: Precisión de los clústeres basados en la matriz de similitud.
    - accuracy_value_X: Precisión de los clústeres basados en los datos originales X.
    """
    np.random.seed(0)
    similarity_matrix = cosine_similarity(X)
    np.fill_diagonal(similarity_matrix, 0)

    predicted_labels_similarity = calculate_kmeans (similarity_matrix, verbose)
    predicted_labels_X = calculate_kmeans (X, verbose)

    labels = Y
    if verbose:
        print(f'trial: {trial}')
        print(f'\t labels: {labels}')
        print(f'\t predicted_labels_similarity: {predicted_labels_similarity}')
        print(f'\t predicted_labels_X: {predicted_labels_X}')

    # Calculate ARI for similarity matrix
    ari_similarity = adjusted_rand_score (labels, predicted_labels_similarity)
    accuracy_value_similarity = accuracy_score (labels, predicted_labels_similarity)

    file_log_similarity = f"./optim_param/{file_prefix}_similarity.txt"
    # Log results including ARI
    log_results(file_log_similarity, trial, trial.params, None, accuracy_value_similarity, None, None, ari_similarity)

    # Calculate ARI for X
    ari_X = adjusted_rand_score(labels, predicted_labels_X)
    accuracy_value_X = accuracy_score(labels, predicted_labels_X)

    file_log_X = f"./optim_param/{file_prefix}_X.txt"
    # Log results including ARI
    log_results(file_log_X, trial, trial.params, None, accuracy_value_X, None, None, ari_X)

    if verbose:
        print(f'\t ARI similarity: {ari_similarity}')
        print(f'\t Accuracy similarity: {accuracy_value_similarity}')

        print(f'\t ARI X: {ari_X}')
        print(f'\t Accuracy X: {accuracy_value_X}')

    return accuracy_value_similarity, accuracy_value_X

def evaluate_clustering_consistency (trial, X, Y, file_prefix, verbose = True):
    """
    Evaluar la consistencia de los clústeres utilizando K-means sobre la matriz de similitud de coseno.

    Args:
    - trial: Objeto de prueba para registrar resultados.
    - X: Datos de entrada para calcular la matriz de similitud de coseno.
    - Y: Etiquetas verdaderas para comparar con los clústeres predichos.
    - file_prefix: Prefijo para los archivos de registro de resultados.
    - verbose: Booleano para imprimir información detallada.

    Returns:
    - accuracy_value_similarity: Precisión de los clústeres basados en la matriz de similitud.
    """
    np.random.seed(0)
    similarity_matrix = cosine_similarity(X)
    np.fill_diagonal(similarity_matrix, 0)

    predicted_labels_similarity = calculate_kmeans (similarity_matrix, verbose)

    labels = Y
    if verbose:
        print(f'trial: {trial}')
        print(f'\t labels: {labels}')
        print(f'\t predicted_labels_similarity: {predicted_labels_similarity}')

    # Calculate ARI for similarity matrix
    ari_similarity = adjusted_rand_score(labels, predicted_labels_similarity)
    accuracy_value_similarity = accuracy_score(labels, predicted_labels_similarity)

    file_log_similarity = f"./optim_param/{file_prefix}_similarity_consistency.txt"
    # Log results including ARI
    log_results(file_log_similarity, trial, trial.params, None, accuracy_value_similarity, None, None, ari_similarity)

    if verbose:
        print(f'\t ARI similarity: {ari_similarity}')
        print(f'\t Accuracy similarity: {accuracy_value_similarity}')

    return accuracy_value_similarity

def train_and_evaluate_model (trial, X, Y, channels = 's', freq = 30, decim = 4,
                               ica = False, features = False, verbose = False):
    config = {
        'seed':0,
        'init_type':trial.suggest_categorical('init_type', ['orthogonal', 'trunc_normal', 'binorm']),
        'init_std':trial.suggest_float('init_std', 0.01, 0.05, log = True),
        'init_mean':0,
        'input_size': X.shape[2],
        'n_internal_units': X.shape[2]*10,
        'spectral_radius': trial.suggest_float('spectral_radius', 0.96, 1.00),
        'leak': trial.suggest_float('leak', 0.38, 0.96),
        'input_scaling': 0.10,
        'nonlinearity': trial.suggest_categorical('nonlinearity', ['relu', 'tanh']),
        'connectivity': trial.suggest_float('connectivity', 0.1, 0.5),
        'noise_level': trial.suggest_float('noise_level', 0.1, 10.00),
        'n_drop': trial.suggest_int('n_drop', 0, 100) if trial.suggest_categorical('n_drop_present', [True, False]) else None,
        'washout':'init',
        'w_ridge_embedding':trial.suggest_float('w_ridge_embedding', 10.0, 20.0, log = True),
        'mts_rep': trial.suggest_categorical('mts_rep', ['reservoir', 'output', 'id','last','mean','state']),
        'bidir':'True', # trial.suggest_categorical('bidir', ['True', 'False']),
        'circle': False,
        'dimred_method':  trial.suggest_categorical('dimred_method', ['tenpca', None]),
        'n_dim': trial.suggest_int('n_dim', 30, 75) if trial.suggest_categorical('dimred_method', ['tenpca', None]) == 'tenpca' else 30,
        'use_input_bias':True,
        'use_input_layer':'True', #trial.suggest_categorical('use_input_layer', ['True', 'False']),
        'readout_type': None,
        'threshold':0.3,
        'svm_kernel': 'linear',
        'svm_gamma': 0.005,
        'svm_C': 5.0,
        'w_ridge': 5.0,
        'mlp_w_l2': 0.01,
        'mlp_num_epochs': 2000,
        'mlp_layout': (10, 10),
        'mlp_batch_size': 32,
        'mlp_learning_rate': 0.01,
        'mlp_learning_rate_type': 'constant',
        'plasticity_synaptic':None, # 'hebb'.'oja', 'covariance','bcm'
        'theta_m':0.01,
        'learning_rate':0.1,
        'plasticity_intrinsic':None, # 'excitability', 'activation_function'
        'new_activation_function':'tanh',
        'excitability_factor':0.01,
        'device': 'cpu'
    }

    config ['bidir']           = True if config ['bidir'] == 'True' else False
    config ['use_input_layer'] = True if config ['use_input_layer'] == 'True' else False

    if config['dimred_method'] is not None:
        config['n_dim'] = min(config['n_dim'], config ['n_internal_units'])

    # Lógica para ajustar washout si n_drop no es None
    if config['n_drop'] is not None:
        config['washout'] = trial.suggest_categorical ('washout', ['init', 'rand'])

    rc_state, input_repr, Y = train_reservoir (config, X, Y, verbose = verbose)

    dt_rc_state = rc_state.reshape (rc_state.shape[0], -1)

    file_prefix_state = f"results_state_{'features_' if features else ''}{channels}_eeg_{'ica' if ica else 'noica'}_{freq}_{decim}"
    file_prefix_input = f"results_inter_{'features_' if features else ''}{channels}_eeg_{'ica' if ica else 'noica'}_{freq}_{decim}"

    accuracy_value_state_similarity, accuracy_value_state_X = evaluate_accuracy (trial, dt_rc_state, Y, channels,
                                                                                 state = True, file_prefix = file_prefix_state, verbose = verbose)
    accuracy_value_input_similarity, accuracy_value_input_X = evaluate_accuracy (trial, input_repr, Y, channels,
                                                                                 state = False, file_prefix = file_prefix_input, verbose = verbose)

    return max (accuracy_value_state_similarity, accuracy_value_input_similarity, accuracy_value_state_X, accuracy_value_input_X)

##################################
#  main script bayes 1
##################################
def create_objective (X, Y, channels = 's', freq = 30, decim = 4,  ica = False, features = False, verbose = False):
    def objective (trial):
        # función para entrenar y evaluar tu modelo
        score = train_and_evaluate_model (trial, X, Y, channels, freq = freq, decim = decim,
                                          ica = ica, features = features, verbose = verbose)
        return score

    # Definir el pruner: en este caso, usando el pruner MedianPruner
    pruner = optuna.pruners.MedianPruner (n_startup_trials = 5, n_warmup_steps = 10)

    return objective, pruner

#############
## El valor de channels puede ser:{'s','z','d','i','eeg'}
## El valor de icaFlag puede ser:{True,False,, }
## El valor de featuresFlag puede ser:{True,False, }

# Función principal
def main (args):
    channels     = args.channels
    icaFlag      = args.icaFlag
    featuresFlag = args.featuresFlag
    freq         = args.freq
    decim        = args.decim
    n_trials     = args.trials
    verbose      = args.verbose

    try:
        X, Y = check_and_load_files (channels, featuresFlag, icaFlag, freq, decim, verbose)
        if verbose: print ("Data loaded successfully.")
    except FileNotFoundError as e:
        print (e)
        return
    try:
        objective, pruner = create_objective (X, Y, channels = channels, freq = freq, decim = decim,  ica = icaFlag, features = featuresFlag, verbose = verbose)
        study             = optuna.create_study (direction = 'maximize', pruner = pruner)
        study.optimize (objective, n_trials = n_trials)

        best_trial = study.best_trial
        with open ("results_pt.txt", "a") as f:
            f.write ("\n---\n")
            f.write ("Mejor configuración:\n")
            f.write (f"{best_trial.params}\n")
            f.write (f"Mejor puntuación (f1_score): {best_trial.value}\n")

        if verbose: print ("Mejor configuración:", best_trial.params)
        if verbose: print ("Mejor puntuación:", best_trial.value)
    except Exception as e:
        print (e)
if __name__ == "__main__":
    if 'ipykernel' in sys.modules:  # Verificar si estamos en un notebook
        # Asignar valores para ejecución en notebook
        args = argparse.Namespace(
            channels     = 'eeg',
            icaFlag      = True,
            featuresFlag = False,
            freq         = 30,
            decim        = 4,
            trials       = 2,
            verbose      = True
        )
    else:
        # Configurar los argumentos de línea de comandos
        parser = argparse.ArgumentParser(description = "Optimización de parámetros con Optuna.")
        parser.add_argument ('--channels', type = str, choices = ['s', 'z', 'd', 'i', 'eeg'], required = True, help = "El valor de channels.")
        parser.add_argument ('--icaFlag', type = bool, choices = [True, False], required = False, help = "El valor de icaFlag.")
        parser.add_argument ('--featuresFlag', type = bool, choices = [True, False], required = False, help = "El valor de featuresFlag.")
        parser.add_argument ('--freq', type = int, default = 30, help = "La frecuencia de muestreo (por defecto es 30).")
        parser.add_argument ('--decim', type = int, default = 4, help = "El factor de decimación (por defecto es 4).")
        parser.add_argument ('--trials', type = int, default = 1, help = "El número de iteraciones (por defecto es 4).")
        parser.add_argument ('--verbose', action = 'store_true', default = True, help = 'Activar trazas')

        # Parsear los argumentos de la línea de comandos
        args = parser.parse_args ()

    # Llamar a la función principal con los argumentos obtenidos
    main(args)

file_RC_y: ./dtset/dt_Younger_eeg_ICA_30_4.npy
file_RC_o: ./dtset/dt_Older_eeg_ICA_30_4.npy


[I 2024-07-14 22:51:58,551] A new study created in memory with name: no-name-52ea45b4-b390-45f4-b619-4e496f55a94c


dataset_RC_y: (23, 39680, 64)
dataset_RC_o: (24, 39680, 64)
Forma de los datos concatenados: (47, 39680, 64)
Forma de las etiquetas concatenadas: (47,)
Data loaded successfully.
config:{'seed': 0, 'init_type': 'orthogonal', 'init_std': 0.03938751119778633, 'init_mean': 0, 'input_size': 64, 'n_internal_units': 640, 'spectral_radius': 0.9826292671395657, 'leak': 0.6897763380528839, 'input_scaling': 0.1, 'nonlinearity': 'tanh', 'connectivity': 0.35317785609255226, 'noise_level': 4.060066620074125, 'n_drop': None, 'washout': 'init', 'w_ridge_embedding': 11.114247598296819, 'mts_rep': 'reservoir', 'bidir': True, 'circle': False, 'dimred_method': None, 'n_dim': 30, 'use_input_bias': True, 'use_input_layer': True, 'readout_type': None, 'threshold': 0.3, 'svm_kernel': 'linear', 'svm_gamma': 0.005, 'svm_C': 5.0, 'w_ridge': 5.0, 'mlp_w_l2': 0.01, 'mlp_num_epochs': 2000, 'mlp_layout': (10, 10), 'mlp_batch_size': 32, 'mlp_learning_rate': 0.01, 'mlp_learning_rate_type': 'constant', 'plasticity_syna

[W 2024-07-14 22:57:18,197] Trial 0 failed with parameters: {'init_type': 'orthogonal', 'init_std': 0.03938751119778633, 'spectral_radius': 0.9826292671395657, 'leak': 0.6897763380528839, 'nonlinearity': 'tanh', 'connectivity': 0.35317785609255226, 'noise_level': 4.060066620074125, 'n_drop_present': False, 'w_ridge_embedding': 11.114247598296819, 'mts_rep': 'reservoir', 'dimred_method': None} because of the following error: TypeError('log_results() takes 7 positional arguments but 8 were given').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-3-5143ae0998bc>", line 355, in objective
    score = train_and_evaluate_model (trial, X, Y, channels, freq = freq, decim = decim,
  File "<ipython-input-3-5143ae0998bc>", line 342, in train_and_evaluate_model
    accuracy_value_state_similarity, accuracy_value_state_X = evaluate_accuracy (trial, dt_rc_sta

trial: <optuna.trial._trial.Trial object at 0x78ee0cb44340>
	 labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1]
	 predicted_labels_similarity: [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 1 1 0 0 0 0 1 1 0]
	 predicted_labels_X: [1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 1]
log_results() takes 7 positional arguments but 8 were given


In [ ]:
def check_and_load_files (channels = 's', ica = True, features = False,
                          freq = 30, decim = 4, verbose = False):

    # Check if the directory exists, if not, raise an exception
    #if not os.path.exists("./dataset"):
        #raise FileNotFoundError("El directorio de datasets no se encuentra disponible.")

     # Name Files EEG data
    file_RC_y = f"./dtset/dt_Younger_{channels}{'_ICA_' if features else '_'}{'features_' if ica else ''}{freq}_{decim}.npy"
    file_RC_o = f"./dtset/dt_Older_{channels}{'_ICA_' if features else '_'}{'features_' if ica else ''}{freq}_{decim}.npy"

    print (f'file_RC_y: {file_RC_y}')
    print (f'file_RC_o: {file_RC_o}')
####################
check_and_load_files (channels = 's', ica = True, features = True,freq = 30, decim = 4, verbose = False)
check_and_load_files (channels = 's', ica = True, features = False,freq = 30, decim = 4, verbose = False)
check_and_load_files (channels = 's', ica = False, features = False,freq = 30, decim = 4, verbose = False)
check_and_load_files (channels = 's', ica = False, features = True,freq = 30, decim = 4, verbose = False)

file_RC_y: ./dtset/dt_Younger_s_ICA_features_30_4.npy
file_RC_o: ./dtset/dt_Older_s_ICA_features_30_4.npy
file_RC_y: ./dtset/dt_Younger_s_features_30_4.npy
file_RC_o: ./dtset/dt_Older_s_features_30_4.npy
file_RC_y: ./dtset/dt_Younger_s_30_4.npy
file_RC_o: ./dtset/dt_Older_s_30_4.npy
file_RC_y: ./dtset/dt_Younger_s_ICA_30_4.npy
file_RC_o: ./dtset/dt_Older_s_ICA_30_4.npy
